In [16]:
"""
Continuous Restricted Boltzmann Machine
14/09/2008 - v. 0.1 by http://imonad.com
"""

from numpy import *
from numpy.random import *
import pylab as p
from scipy import stats, mgrid, c_, reshape, random, rot90

# import psyco
# psyco.full()


class RBM:
    def __init__(self, nvis, nhid):
        self.sig = 0.2
        self.epsW = 0.5
        self.epsA  = 0.5
        self.nvis = nvis
        self.nhid = nhid
        self.Ndat = 500
        self.cost = 0.00001
        self.moment = 0.90
        self.Svis0 = zeros( nvis+1, dtype=float32)
        self.Svis0[-1] = 1.0
        self.Svis = zeros( nvis+1, dtype=float32)
        self.Svis[-1] = 1.0
        self.Shid = zeros( nhid+1, dtype=float32)
        self.W  = standard_normal((nvis+1, nhid+1))/10
        self.dW = standard_normal((nvis+1, nhid+1))/1000
        self.Avis  =  0.1*ones( nvis+1, dtype=float32)
        self.Ahid  =  ones( nhid+1, dtype=float32)
        self.dA = zeros( nvis+1, dtype=float32)
        self.dat = self.genData()

    def genData(self):
        c1 = 0.5
        r1 = 0.4
        r2 = 0.3
        # generate enough data to filter
        N = 20* self.Ndat
        X = array(random_sample(N))
        Y = array(random_sample(N))
        X1 = X[(X-c1)*(X-c1) + (Y-c1)*(Y-c1) < r1*r1]
        Y1 = Y[(X-c1)*(X-c1) + (Y-c1)*(Y-c1) < r1*r1]
        X2 = X1[(X1-c1)*(X1-c1) + (Y1-c1)*(Y1-c1) > r2*r2]
        Y2 = Y1[(X1-c1)*(X1-c1) + (Y1-c1)*(Y1-c1) > r2*r2]
        X3 = X2[ abs(X2-Y2)>0.05 ]
        Y3 = Y2[ abs(X2-Y2)>0.05 ]
        #X3 = X2[ X2-Y2>0.15 ]
        #Y3 = Y2[ X2-Y2>0.15]
        X4=zeros( self.Ndat, dtype=float32)
        Y4=zeros( self.Ndat, dtype=float32)
        for i in range(self.Ndat):
            if (X3[i]-Y3[i]) >0.05:
                X4[i] = X3[i] + 0.08
                Y4[i] = Y3[i] + 0.18
            else:
                X4[i] = X3[i] - 0.08
                Y4[i] = Y3[i] - 0.18
        print("X", size(X3[0:self.Ndat]), "Y", size(Y3))
#         print(vstack((X4[0:self.Ndat],Y4[0:self.Ndat]))[0])
        return(vstack((X4[0:self.Ndat],Y4[0:self.Ndat])))

    # Sigmoidal Function
    def sigFun(self, X, A):
        lo = 0.0
        hi = 1.0
        return(lo + (hi-lo)/(1.0 + exp(-A*X)))

    # visible=0, hidden=1
    def activ(self, who):
        if(who==0):
            self.Svis = dot(self.W, self.Shid) + self.sig*standard_normal(self.nvis+1)         
            self.Svis = self.sigFun(self.Svis, self.Avis)
            self.Svis[-1] = 1.0 # bias
        if(who==1):
            self.Shid = dot(self.Svis, self.W) + self.sig*standard_normal(self.nhid+1)
            self.Shid = self.sigFun(self.Shid, self.Ahid)
            self.Shid[-1] = 1.0 # bias

    def wview(self):
        import pylab as p
        p.plot(range(size(self.W[2])),self.W[2], 'bo')
        p.show()

    def learn(self, epochmax):
        Err = zeros( epochmax, dtype=float32)
        E = zeros( epochmax, dtype=float32)
        self.stat = zeros( epochmax, dtype=float32)
        self.stat2 = zeros( epochmax, dtype=float32)
        ksteps = 1
        
        for epoch in range(1,epochmax):
            wpos = zeros( (self.nvis+1, self.nhid+1), dtype=float32)
            wneg = zeros( (self.nvis+1, self.nhid+1), dtype=float32)
            apos = zeros( self.nhid+1, dtype=float32)
            aneg = zeros( self.nhid+1, dtype=float32)
                
            if(epoch>0):
                ksteps=50
            if(epoch>1000):
                ksteps=(epoch-epoch%100)/100+40
            self.ksteps = ksteps
            
            for point in range(self.Ndat):
                #print(self.dat[:][point])
                self.Svis0[0:2] = self.dat[:,point]
                self.Svis = self.Svis0
                # positive phase
                self.activ(1)
                wpos = wpos + outer(self.Svis, self.Shid)
                apos = apos + self.Shid*self.Shid
                # negative phase
                self.activ(0)
                self.activ(1)
                
                
                for recstep in range(ksteps): 
                    self.activ(0)
                    self.activ(1)

                tmp = outer(self.Svis, self.Shid)
                wneg = wneg + tmp
                aneg = aneg + self.Shid*self.Shid
                
                
                delta = self.Svis0[0:2]-self.Svis[0:2]
                # statistics
                Err[epoch] = Err[epoch] + sum(delta*delta)
                E[epoch] =E[epoch] -sum( dot(self.W.T, tmp) )
                

            
            self.dW = self.dW*self.moment + self.epsW * ((wpos-wneg)/size(self.dat) - self.cost*self.W)
            self.W = self.W + self.dW
            self.Ahid = self.Ahid + self.epsA*(apos-aneg)/(size(self.dat)*self.Ahid*self.Ahid)

            Err[epoch] = Err[epoch]/(self.nvis*size(self.dat))
            E[epoch] = E[epoch]/size(self.dat)
            if (epoch%100==0) or (epoch==epochmax) or (epoch==1):
                print("epoch:", epoch, "err:", round_(Err[epoch], 6), "ksteps:", ksteps)
            self.stat[epoch] = self.W[0,0]
            self.stat2[epoch] = self.Ahid[0]
        self.Err = Err
        self.E   = E
        

    def reconstruct(self, Npoint, Nsteps):
        X = array(random_sample(Npoint))
        Y = array(random_sample(Npoint))
        datnew = vstack((X, Y))
        self.datout = zeros( (2,Npoint), dtype=float32)
        for point in range(Npoint):
            self.Svis[0:2] = datnew[:,point]
            for recstep in range(Nsteps): 
                self.activ(1)
                self.activ(0)
        
            self.datout[:,point] = self.Svis[0:2]
            
    def contour(self, p, dat):
        X, Y = mgrid[0.0:1.0:100j, 0.0:1.0:100j]
        positions = c_[X.ravel(), Y.ravel()]
        val           = c_[dat[0,:], dat[1,:]]
        kernel = stats.kde.gaussian_kde(val.T)
        Z = reshape(kernel(positions.T).T, X.T.shape)
        p.imshow(     rot90(Z) , cmap=p.cm.YlGnBu, extent=[0, 1, 0, 1])
        p.plot(dat[0,:], dat[1,:], 'r.')
        p.axis([0.0, 1.0, 0.0, 1.0])
    

        

if __name__ == "__main__":
    
    seed(12345)
    rbm = RBM(2,8)
    rbm.learn(4000)
    kkk=0
    
    p.figure(1)
    p.plot(range(size(rbm.E)),rbm.E, 'b+')

    p.figure(2)
    p.plot(range(size(rbm.Err)),rbm.Err, 'r.')

    p.figure(3)
    if kkk==1:
        p.plot(rbm.dat[0,:],rbm.dat[1,:], 'bo')
        p.axis([0.0, 1.0, 0.0, 1.0])
    else:
        rbm.contour(p, rbm.dat)
        p.savefig("dat.png",dpi=100)


    rbm.reconstruct(rbm.Ndat, 1)
    p.figure(4)
    if kkk==1:
        p.plot(rbm.datout[0,:],rbm.datout[1,:], 'b.')
        p.axis([0.0, 1.0, 0.0, 1.0])
    else:
        rbm.contour(p, rbm.datout) 
    
    rbm.reconstruct(rbm.Ndat, 20)
    p.figure(5)
    if kkk==1:
        p.plot(rbm.datout[0,:],rbm.datout[1,:], 'b.')
        p.axis([0.0, 1.0, 0.0, 1.0])
    else:
        rbm.contour(p, rbm.datout)
    
    
    rbm.reconstruct(rbm.Ndat, rbm.ksteps)
    p.figure(6)
    if kkk==1:
        p.plot(rbm.datout[0,:],rbm.datout[1,:], 'b.')
        p.axis([0.0, 1.0, 0.0, 1.0])
    else:
        rbm.contour(p, rbm.datout)
        p.savefig("reconstruct.png",dpi=100)

    p.figure(7)
    p.plot(range(size(rbm.stat)), rbm.stat, "b.")

    p.figure(8)
    p.plot(range(size(rbm.stat2)), rbm.stat2, "b.")

    print(around(rbm.W,5))
    print(rbm.Ahid)

    p.show()



X 500 Y 2055
[0.8728007 0.5209726 1.       ]
[0.8728007 0.5209726 1.       ]
[0.3324926  0.37589872 1.        ]
[0.3324926  0.37589872 1.        ]
[0.14657505 0.0972025  1.        ]
[0.14657505 0.0972025  1.        ]
[0.84873545 0.4186513  1.        ]
[0.84873545 0.4186513  1.        ]
[0.9063804  0.83182514 1.        ]
[0.9063804  0.83182514 1.        ]
[0.06294502 0.48037153 1.        ]
[0.06294502 0.48037153 1.        ]
[0.76934135 0.40921712 1.        ]
[0.76934135 0.40921712 1.        ]
[0.93662584 0.5503505  1.        ]
[0.93662584 0.5503505  1.        ]
[0.43320626 0.6767758  1.        ]
[0.43320626 0.6767758  1.        ]
[0.7623564  0.37933066 1.        ]
[0.7623564  0.37933066 1.        ]
[0.40954039 0.6407464  1.        ]
[0.40954039 0.6407464  1.        ]
[0.334351   0.65552264 1.        ]
[0.334351   0.65552264 1.        ]
[0.07603859 0.18778512 1.        ]
[0.07603859 0.18778512 1.        ]
[0.53399485 0.32138944 1.        ]
[0.53399485 0.32138944 1.        ]
[0.3340918  0

[0.46239588 0.35885897 1.        ]
[0.46239588 0.35885897 1.        ]
[0.55889934 0.311181   1.        ]
[0.55889934 0.311181   1.        ]
[0.13561249 0.43106225 1.        ]
[0.13561249 0.43106225 1.        ]
[0.92018247 0.7415917  1.        ]
[0.92018247 0.7415917  1.        ]
[0.07297806 0.25343207 1.        ]
[0.07297806 0.25343207 1.        ]
[0.9561416  0.79181373 1.        ]
[0.9561416  0.79181373 1.        ]
[0.81570214 0.38715705 1.        ]
[0.81570214 0.38715705 1.        ]
[0.372224  0.4047163 1.       ]
[0.372224  0.4047163 1.       ]
[0.14472306 0.60451126 1.        ]
[0.14472306 0.60451126 1.        ]
[0.84974235 0.858465   1.        ]
[0.84974235 0.858465   1.        ]
[0.89285237 0.62035364 1.        ]
[0.89285237 0.62035364 1.        ]
[0.34099463 0.63803035 1.        ]
[0.34099463 0.63803035 1.        ]
[0.21181804 0.5541876  1.        ]
[0.21181804 0.5541876  1.        ]
[0.34431544 0.64838636 1.        ]
[0.34431544 0.64838636 1.        ]
[0.922356  0.8073947 1.   

[0.58918345 0.28523636 1.        ]
[0.58918345 0.28523636 1.        ]
[0.14822935 0.48327968 1.        ]
[0.14822935 0.48327968 1.        ]
[0.60325783 0.36400703 1.        ]
[0.60325783 0.36400703 1.        ]
[0.47629383 0.29618812 1.        ]
[0.47629383 0.29618812 1.        ]
[0.69279057 0.3134072  1.        ]
[0.69279057 0.3134072  1.        ]
[0.9404187 0.6196024 1.       ]
[0.9404187 0.6196024 1.       ]
[0.47586596 0.3358903  1.        ]
[0.47586596 0.3358903  1.        ]
[0.41365364 0.42743292 1.        ]
[0.41365364 0.42743292 1.        ]
[0.88317704 0.5398271  1.        ]
[0.88317704 0.5398271  1.        ]
[0.5462746  0.36963788 1.        ]
[0.5462746  0.36963788 1.        ]
[0.5085283 0.6174032 1.       ]
[0.5085283 0.6174032 1.       ]
[0.06087504 0.27638656 1.        ]
[0.06087504 0.27638656 1.        ]
[0.4319922 0.3915392 1.       ]
[0.4319922 0.3915392 1.       ]
[0.35248193 0.6798559  1.        ]
[0.35248193 0.6798559  1.        ]
[0.94902843 0.60740274 1.        ]
[0.

epoch: 1 err: 0.032999 ksteps: 50
[0.8728007 0.5209726 1.       ]
[0.8728007 0.5209726 1.       ]
[0.3324926  0.37589872 1.        ]
[0.3324926  0.37589872 1.        ]
[0.14657505 0.0972025  1.        ]
[0.14657505 0.0972025  1.        ]
[0.84873545 0.4186513  1.        ]
[0.84873545 0.4186513  1.        ]
[0.9063804  0.83182514 1.        ]
[0.9063804  0.83182514 1.        ]
[0.06294502 0.48037153 1.        ]
[0.06294502 0.48037153 1.        ]
[0.76934135 0.40921712 1.        ]
[0.76934135 0.40921712 1.        ]
[0.93662584 0.5503505  1.        ]
[0.93662584 0.5503505  1.        ]
[0.43320626 0.6767758  1.        ]
[0.43320626 0.6767758  1.        ]
[0.7623564  0.37933066 1.        ]
[0.7623564  0.37933066 1.        ]
[0.40954039 0.6407464  1.        ]
[0.40954039 0.6407464  1.        ]
[0.334351   0.65552264 1.        ]
[0.334351   0.65552264 1.        ]
[0.07603859 0.18778512 1.        ]
[0.07603859 0.18778512 1.        ]
[0.53399485 0.32138944 1.        ]
[0.53399485 0.32138944 1.  

[0.788209  0.3998531 1.       ]
[0.788209  0.3998531 1.       ]
[0.06708024 0.32651773 1.        ]
[0.06708024 0.32651773 1.        ]
[0.412203  0.3211839 1.       ]
[0.412203  0.3211839 1.       ]
[0.42466906 0.691369   1.        ]
[0.42466906 0.691369   1.        ]
[0.88881195 0.7791799  1.        ]
[0.88881195 0.7791799  1.        ]
[0.3591186 0.6780023 1.       ]
[0.3591186 0.6780023 1.       ]
[0.45884922 0.3135321  1.        ]
[0.45884922 0.3135321  1.        ]
[0.473627  0.7135546 1.       ]
[0.473627  0.7135546 1.       ]
[0.21814719 0.64846075 1.        ]
[0.21814719 0.64846075 1.        ]
[0.64567965 0.35057703 1.        ]
[0.64567965 0.35057703 1.        ]
[0.73962367 0.3234181  1.        ]
[0.73962367 0.3234181  1.        ]
[0.9718322 0.7303341 1.       ]
[0.9718322 0.7303341 1.       ]
[0.6727174  0.35625505 1.        ]
[0.6727174  0.35625505 1.        ]
[0.23841669 0.6368864  1.        ]
[0.23841669 0.6368864  1.        ]
[0.30153206 0.69294435 1.        ]
[0.30153206 0.6

...

[0.8480889  0.43216118 1.        ]
[0.8480889  0.43216118 1.        ]
[0.8778047  0.42477652 1.        ]
[0.8778047  0.42477652 1.        ]
[0.33236268 0.67439145 1.        ]
[0.33236268 0.67439145 1.        ]
[0.47649887 0.3697464  1.        ]
[0.47649887 0.3697464  1.        ]
[0.10687476 0.19018042 1.        ]
[0.10687476 0.19018042 1.        ]
[0.87851506 0.8073077  1.        ]
[0.87851506 0.8073077  1.        ]
[0.08891282 0.42392108 1.        ]
[0.08891282 0.42392108 1.        ]
[0.7621773  0.35704663 1.        ]
[0.7621773  0.35704663 1.        ]
[0.12119471 0.21907467 1.        ]
[0.12119471 0.21907467 1.        ]
[0.77189773 0.44206896 1.        ]
[0.77189773 0.44206896 1.        ]
[0.7603717  0.36940703 1.        ]
[0.7603717  0.36940703 1.        ]
[0.3404948  0.36750576 1.        ]
[0.3404948  0.36750576 1.        ]
[0.19584268 0.63841206 1.        ]
[0.19584268 0.63841206 1.        ]
[0.92114806 0.66374207 1.        ]
[0.92114806 0.66374207 1.        ]
[0.11642437 0.241729

[0.8895095 0.5427155 1.       ]
[0.8895095 0.5427155 1.       ]
[0.97638535 0.728248   1.        ]
[0.97638535 0.728248   1.        ]
[0.9507462  0.68619627 1.        ]
[0.9507462  0.68619627 1.        ]
[0.6152916 0.3029692 1.       ]
[0.6152916 0.3029692 1.       ]
[0.07254107 0.35449278 1.        ]
[0.07254107 0.35449278 1.        ]
[0.8645105 0.4804677 1.       ]
[0.8645105 0.4804677 1.       ]
[0.13119617 0.5855112  1.        ]
[0.13119617 0.5855112  1.        ]
[0.9216514 0.5304907 1.       ]
[0.9216514 0.5304907 1.       ]
[0.02645149 0.275066   1.        ]
[0.02645149 0.275066   1.        ]
[0.12008723 0.24042775 1.        ]
[0.12008723 0.24042775 1.        ]
[0.0215256  0.33806494 1.        ]
[0.0215256  0.33806494 1.        ]
[0.13564374 0.53643256 1.        ]
[0.13564374 0.53643256 1.        ]
[0.4997733 0.3334415 1.       ]
[0.4997733 0.3334415 1.       ]
[0.05158821 0.37177575 1.        ]
[0.05158821 0.37177575 1.        ]
[0.5019272 0.6324767 1.       ]
[0.5019272 0.63247

[0.85937643 0.8569931  1.        ]
[0.85937643 0.8569931  1.        ]
[0.29914364 0.64832914 1.        ]
[0.29914364 0.64832914 1.        ]
[0.20907709 0.6370835  1.        ]
[0.20907709 0.6370835  1.        ]
[0.42072862 0.3373857  1.        ]
[0.42072862 0.3373857  1.        ]
[0.9049496 0.8443216 1.       ]
[0.9049496 0.8443216 1.       ]
[0.03782424 0.26941442 1.        ]
[0.03782424 0.26941442 1.        ]
[0.95052755 0.69108635 1.        ]
[0.95052755 0.69108635 1.        ]
[0.3955842 0.7089527 1.       ]
[0.3955842 0.7089527 1.       ]
[0.48072892 0.35901934 1.        ]
[0.48072892 0.35901934 1.        ]
[0.88652354 0.65054554 1.        ]
[0.88652354 0.65054554 1.        ]
[0.87473345 0.46896905 1.        ]
[0.87473345 0.46896905 1.        ]
[0.8260697  0.40623114 1.        ]
[0.8260697  0.40623114 1.        ]
[0.36982793 0.63484704 1.        ]
[0.36982793 0.63484704 1.        ]
[0.6076809  0.35184032 1.        ]
[0.6076809  0.35184032 1.        ]
[0.44613972 0.34644637 1.       

[0.53399485 0.32138944 1.        ]
[0.53399485 0.32138944 1.        ]
[0.3340918  0.64470094 1.        ]
[0.3340918  0.64470094 1.        ]
[0.75174874 0.37181374 1.        ]
[0.75174874 0.37181374 1.        ]
[0.9188657 0.5929782 1.       ]
[0.9188657 0.5929782 1.       ]
[0.8595262  0.41054162 1.        ]
[0.8595262  0.41054162 1.        ]
[0.874201  0.7435325 1.       ]
[0.874201  0.7435325 1.       ]
[0.3768695 0.6660423 1.       ]
[0.3768695 0.6660423 1.       ]
[0.926943   0.74515015 1.        ]
[0.926943   0.74515015 1.        ]
[0.29809955 0.62278897 1.        ]
[0.29809955 0.62278897 1.        ]
[0.47467148 0.63591313 1.        ]
[0.47467148 0.63591313 1.        ]
[0.70363694 0.32946697 1.        ]
[0.70363694 0.32946697 1.        ]
[0.8128631  0.42654365 1.        ]
[0.8128631  0.42654365 1.        ]
[0.16922505 0.59642136 1.        ]
[0.16922505 0.59642136 1.        ]
[0.17541173 0.5969224  1.        ]
[0.17541173 0.5969224  1.        ]
[0.12122424 0.51913    1.        ]
[0.

KeyboardInterrupt: 